In [ ]:
import pandas as pd
from openai import OpenAI
import time
import os
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import json
import random
import re

In [57]:
with open('questions_without_material.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

random.seed(42)
random_sample = random.sample(data, 260)
    
   
with open('random_sample_260.json', 'w', encoding='utf-8') as f:
    json.dump(random_sample, f, ensure_ascii=False, indent=2)

In [ ]:
with open('random_sample_260.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data

In [21]:
chunk_size = 52
total_chunks = 5

for i in range(total_chunks):
    start_index = i * chunk_size
    end_index = start_index + chunk_size
    chunk = data[start_index:end_index]
    
    output_filename = f'диссертация_разметка/данные/data_{i+1}.json'
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(chunk, f, ensure_ascii=False, indent=4)
    
    print(f'Создан файл {output_filename} с {len(chunk)} вопросами')

Создан файл диссертация_разметка/данные/data_1.json с 52 вопросами
Создан файл диссертация_разметка/данные/data_2.json с 52 вопросами
Создан файл диссертация_разметка/данные/data_3.json с 52 вопросами
Создан файл диссертация_разметка/данные/data_4.json с 52 вопросами
Создан файл диссертация_разметка/данные/data_5.json с 52 вопросами


## Ответы на вопросы

### 1. google/gemma-3-27b-it

In [22]:
with open('диссертация_разметка/данные/data_1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data[1]

{'id': 336,
 'Пакет': 'Балтийский берег. IV тур',
 '№': 6.0,
 'Вопрос': 'В одной книге, посвящённой истории медицины, приводится высказывание: «ОН – нянечка для бедняка». В девятнадцатом веке ЕГО действительно могли позволить себе даже представители рабочего класса. Назовите ЕГО.',
 'Ответ': 'опиум',
 'Зачет': 'лауданум.',
 'Комментарий': 'вероятно, авторы книги отсылают к известной цитате «религия – опиум для народа». Чтобы успокоить кричащих младенцев, в девятнадцатом веке могли использовать опиумные настойки.',
 'Источник': None,
 'Авторы': 'Мария Подрядчикова (Волгоград)',
 'Доля взятия': '674/1032',
 'Процент взятия': 65.0,
 'Год': '2019_2020'}

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai",
)

results = []

for i, item in enumerate(data):
    prompt = f"""Ты участвуешь в интеллектуальной игре. Пожалуйста, кратко порассуждай над следующим вопросом и дай ответ.

Вопрос: {item['Вопрос']}

Выведи рассуждение и ответ в формате JSON:
{{
    "reasoning": "твои рассуждения здесь",
    "answer": "ответ здесь"
}}"""

    try:
        chat_completion = openai.chat.completions.create(
            model="google/gemma-3-27b-it",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            seed=1,
            max_tokens=10000
        )

        
        content = chat_completion.choices[0].message.content
        
        results.append({
            "model": "gemma-3",
            "id": item["id"],
            "question": item["Вопрос"],
            "raw_model_response": content
        })
        
        #print(f"\n--- Вопрос {i+1} ---\n{content}")
        
    except Exception as e:
        results.append({
            "model": "gemma-3",
            "id": item["id"],
            "question": item["Вопрос"],
            "error": str(e)
        })
        print(f"Ошибка при обработке вопроса {i+1}: {e}")

    time.sleep(1.0)

with open('диссертация_разметка/model_1_answers_raw.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

### 2. Qwen/QwQ-32B

In [11]:
with open('диссертация_разметка/данные/data_2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data[1]

{'id': 2077,
 'Пакет': 'Турнир «12 граней-2022». Вторая игра',
 '№': 34.0,
 'Вопрос': 'Один писатель сравнил черноту ЧАДСКОГО АДА с влажно-блестящим живым Изидиным зрачком. Какие два слова мы заменили словами «ЧАДСКИЙ АД»?',
 'Ответ': 'нильский ил',
 'Зачет': 'ил Нила.',
 'Комментарий': 'по мнению Дмитрия Мережковского, весь мир для Египтян - чернота нильского ила и краснота мёртвых песков.',
 'Источник': None,
 'Авторы': 'Василий Корягин (Тольятти), Дмитрий Балашов (Тольятти)',
 'Доля взятия': '26/119',
 'Процент взятия': 22.0,
 'Год': '2022_2023'}

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai",
)

results = []

for i, item in enumerate(data):
    prompt = f"""Ты участвуешь в интеллектуальной игре. Пожалуйста, кратко порассуждай над следующим вопросом и дай ответ.

Вопрос: {item['Вопрос']}

Выведи рассуждение и ответ в формате JSON:
{{
    "reasoning": "твои рассуждения здесь",
    "answer": "ответ здесь"
}}"""

    try:
        chat_completion = openai.chat.completions.create(
            model="Qwen/QwQ-32B",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            seed=1,
            max_tokens=3000
        )

        content = chat_completion.choices[0].message.content
        
        results.append({
            "model": "QwQ",
            "id": item["id"],
            "question": item["Вопрос"],
            "raw_model_response": content
        })
        
        #print(f"\n--- Вопрос {i+1} ---\n{content}")
        
    except Exception as e:
        results.append({
            "model": "QwQ",
            "id": item["id"],
            "question": item["Вопрос"],
            "error": str(e)
        })
        print(f"Ошибка при обработке вопроса {i+1}: {e}")

    time.sleep(1.0)

with open('диссертация_разметка/model_2_answers_raw.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

### 3. microsoft/Phi-4-multimodal-instruct

In [3]:
with open('диссертация_разметка/данные/data_3.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data[1]

{'id': 2297,
 'Пакет': 'Лига Сибири. V тур',
 '№': 2.0,
 'Вопрос': 'На международных соревнованиях по шахматам Оскару Роберто Панно зачастую лишь немного не хватало до первого места. Посвящённая Панно и его соотечественникам книга называется «ТАКОЕ поколение». Ответьте, какую страну представляли эти шахматисты, а также какое слово мы заменили словом ТАКОЕ.',
 'Ответ': 'Аргентина, серебряное.',
 'Зачет': None,
 'Комментарий': 'в книге «Серебряное поколение» рассказывается об аргентинских шахматистах, занимавших вторые места на шахматных олимпиадах. Название страны «Аргентина» образовано от корня со значением «серебро».',
 'Источник': 'https://en.chessbase.com/post/argentina-silver-generation',
 'Авторы': 'Антон Саксонов (Санкт-Петербург)',
 'Доля взятия': '165/280',
 'Процент взятия': 59.0,
 'Год': '2022_2023'}

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai",
)

results = []

for i, item in enumerate(data):
    prompt = f"""Ты участвуешь в интеллектуальной игре. Пожалуйста, кратко порассуждай над следующим вопросом и дай ответ.

Вопрос: {item['Вопрос']}

Выведи рассуждение и ответ в формате JSON:
{{
    "reasoning": "твои рассуждения здесь",
    "answer": "ответ здесь"
}}"""

    try:
        chat_completion = openai.chat.completions.create(
            model="microsoft/Phi-4-multimodal-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            seed=1,
            max_tokens=5000
        )

        content = chat_completion.choices[0].message.content
        
        results.append({
            "model": "Phi-4",
            "id": item["id"],
            "question": item["Вопрос"],
            "raw_model_response": content
        })
        
        #print(f"\n--- Вопрос {i+1} ---\n{content}")
        
    except Exception as e:
        results.append({
            "model": "Phi-4",
            "id": item["id"],
            "question": item["Вопрос"],
            "error": str(e)
        })
        print(f"Ошибка при обработке вопроса {i+1}: {e}")

    time.sleep(1.0)

with open('диссертация_разметка/model_3_answers_raw.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

### 4. meta-llama/Llama-4-Scout-17B-16E-Instruct

In [5]:
with open('диссертация_разметка/данные/data_4.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data[1]

{'id': 484,
 'Пакет': 'Балтийский Берег. II тур.',
 '№': 1.0,
 'Вопрос': 'Коч – северное парусное судно, имевшее двойную обшивку и ставшее прообразом современного ледокола. Слово «коч» происходит от диалектного новгородского слова со значением «ОНА». ОНА входит в название блюда. Какого?',
 'Ответ': 'селёдка под шубой',
 'Зачет': 'точный ответ.',
 'Комментарий': 'название «коч» происходит от диалектного новгородского «ко́ца» – шуба, что связано с обшивкой судна, укрепляющей его борта и защищающей от давления льда, да и просто от ударов.',
 'Источник': 'http://flibusta.is/b/500678/read',
 'Авторы': 'Александр Руденко (Минск)',
 'Доля взятия': '832/963',
 'Процент взятия': 86.0,
 'Год': '2019_2020'}

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai",
)

results = []

for i, item in enumerate(data):
    prompt = f"""Ты участвуешь в интеллектуальной игре. Пожалуйста, кратко порассуждай над следующим вопросом и дай ответ.

Вопрос: {item['Вопрос']}

Выведи рассуждение и ответ в формате JSON:
{{
    "reasoning": "твои рассуждения здесь",
    "answer": "ответ здесь"
}}"""

    try:
        chat_completion = openai.chat.completions.create(
            model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            seed=1,
            max_tokens=5000
        )

        content = chat_completion.choices[0].message.content
        
        results.append({
            "model": "Llama-4",
            "id": item["id"],
            "question": item["Вопрос"],
            "raw_model_response": content
        })
        
        #print(f"\n--- Вопрос {i+1} ---\n{content}")
        
    except Exception as e:
        results.append({
            "model": "Llama-4",
            "id": item["id"],
            "question": item["Вопрос"],
            "error": str(e)
        })
        print(f"Ошибка при обработке вопроса {i+1}: {e}")

    time.sleep(1.5)
    
with open('диссертация_разметка/model_4_answers_raw.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

### 5. Qwen/Qwen3-32B

In [9]:
with open('диссертация_разметка/данные/data_5.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

data[1]

{'id': 2625,
 'Пакет': 'Лига вузов. II тур',
 '№': 6.0,
 'Вопрос': 'Внимание, в вопросе есть замена. Древняя Япония переняла многие элементы из культуры древнего Китая, однако часто они достигали Японии уже в изменённом виде. Говоря об этом, ведущий одного подкаста сравнивает древнюю Корею с НИМ. Некоторые англоязычные дети называют ЕГО «секретным сообщением». Назовите ЕГО двумя словами.',
 'Ответ': 'испорченный телефон.',
 'Зачет': 'сломанный телефон; по смыслу.',
 'Комментарий': 'Корея располагается между Китаем и Японией. Многие китайские обычаи достигали Японии уже в видоизмененном корейцами виде, словно слова, прошедшие через испорченный телефон. В английском языке игру «испорченный телефон» чаще всего называют «chinese whispers» [чайни́з ви́сперс] — «китайский шёпот», но существует ряд альтернативных названий, например, «secret message» [сикрет мэссадж].',
 'Источник': 'https://en.wikipedia.org/wiki/Chinese_whispers',
 'Авторы': 'Валентин Копочель (Минск)',
 'Доля взятия': '397/6

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai",
)

results = []

for i, item in enumerate(data):
    prompt = f"""Ты участвуешь в интеллектуальной игре. Пожалуйста, кратко порассуждай над следующим вопросом и дай ответ.

Вопрос: {item['Вопрос']}

Выведи рассуждение и ответ в формате JSON:
{{
    "reasoning": "твои рассуждения здесь",
    "answer": "ответ здесь"
}}"""

    try:
        chat_completion = openai.chat.completions.create(
            model="Qwen/Qwen3-32B",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            seed=1,
            max_tokens=3000
        )

        content = chat_completion.choices[0].message.content
        
        results.append({
            "model": "Qwen3",
            "id": item["id"],
            "question": item["Вопрос"],
            "raw_model_response": content
        })
        
        print(f"\n--- Вопрос {i+1} ---\n{content}")
        
    except Exception as e:
        results.append({
            "model": "Qwen3",
            "id": item["id"],
            "question": item["Вопрос"],
            "error": str(e)
        })
        print(f"Ошибка при обработке вопроса {i+1}: {e}")

    time.sleep(1.0)
    
with open('диссертация_разметка/model_5_answers_raw.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

### Постобработка для всех ответов

In [ ]:
def extract_answer(raw_response):
    json_match = re.search(r'\{[\s\S]*\}', raw_response)
    if not json_match:
        return "Ответ не дан"
    
    try:
        data = json.loads(json_match.group(0))
        return data.get("answer", "Ключ 'answer' не найден")
    except json.JSONDecodeError:
        return "Ответ не дан"

processed_results = []

for i in range(1, 6):
    input_filename = f'диссертация_разметка/model_{i}_answers_raw.json'
    try:
        with open(input_filename, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)

        for item in raw_data:
            processed_results.append({
                "model": item.get("model", f"model_{i}"),
                "id": item["id"],
                "question": item["question"],
                "answer": extract_answer(item["raw_model_response"])
            })
            
    except Exception as e:
        print(f"Ошибка в файле {input_filename}: {str(e)}")

with open('диссертация_разметка/answers_260.json', 'w', encoding='utf-8') as f:
    json.dump(processed_results, f, ensure_ascii=False, indent=2)

## Оценка ответов

### 6. deepseek-ai/DeepSeek-R1

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai"
)

with open('random_sample_260.json') as f:
    correct_data = json.load(f)

with open('диссертация_разметка/answers_260.json') as f:
    model_responses = json.load(f)


model_responses_dict = {item['id']: item for item in model_responses}

model_results = []
for item in correct_data:
    question_id = item['id']
    model_response = model_responses_dict.get(question_id)
    
    if not model_response:
        print(f"Ответ для вопроса с ID {question_id} не найден")
        continue
        
    question = item['Вопрос']
    model_answer = model_response['answer'].strip()
    correct_answer = item['Ответ']
    zachet = item['Зачет'].split(',') if item['Зачет'] else []

    prompt = f"""Ты эксперт, оценивающий ответы в интеллектуальной игре. Оцени ответ:
       true - ответ правильный, false - ответ неправильный.
       
    Вопрос: {question}
    Ответ: {model_answer}

    Правильный ответ: {correct_answer}
    Допустимые варианты ответа, которые тоже засчитываются как верные: {', '.join(zachet) if zachet else 'Точный ответ'}

    Верни JSON:
    {{
    "is_correct": true или false,
    "comment": "Краткое объяснение оценки"
    }}"""
        
    try:
        chat_completion = openai.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            seed=2,
            temperature=0,
            max_tokens=5000
        )
            
        response_text = chat_completion.choices[0].message.content
        
        json_str = re.search(r'\{.*\}', response_text, re.DOTALL)
        reasoning_match = re.search(r'<think>(.*?)</think>', response_text, re.DOTALL)
        
        if json_str:
            evaluation = json.loads(json_str.group())
            reasoning = reasoning_match.group(1).strip() if reasoning_match else ""
        else:
            evaluation = {
                "is_correct": False,
                "comment": "Не удалось распознать оценку"
            }
            reasoning = response_text
            
    except Exception as e:
        evaluation = {
            "is_correct": False,
            "comment": f"Ошибка при оценке: {str(e)}",
            "raw": response_text
        }
        reasoning = "Произошла ошибка при запросе к API"

    model_results.append({
        "evaluation_model": "DeepSeek-R1",
        "answer_model": model_responses_dict.get(question_id)['model'],
        "question_id": question_id,
        "question": question,
        "model_answer": model_answer,
        "correct_answer": correct_answer,
        "zachet": ', '.join(zachet) if zachet else 'Точный ответ',
        "evaluation": evaluation,
        "reasoning": reasoning
    })

    time.sleep(0.5) 

with open('диссертация_разметка/model_6_evaluation.json', 'w', encoding='utf-8') as f:
    json.dump(model_results, f, ensure_ascii=False, indent=2)

print("Оценка всех ответов завершена!")

### 7. deepseek-ai/DeepSeek-V3-0324

In [ ]:
openai = OpenAI(
    api_key="",
    base_url="https://api.deepinfra.com/v1/openai"
)

with open('random_sample_260.json') as f:
    correct_data = json.load(f)

with open('диссертация_разметка/answers_260.json') as f:
    model_responses = json.load(f)
    

model_responses_dict = {item['id']: item for item in model_responses}

model_results = []
for item in correct_data:
    question_id = item['id']
    model_response = model_responses_dict.get(question_id)
    
    if not model_response:
        print(f"Ответ для вопроса с ID {question_id} не найден")
        continue
        
    question = item['Вопрос']
    model_answer = model_response['answer'].strip()
    correct_answer = item['Ответ']
    zachet = item['Зачет'].split(',') if item['Зачет'] else []

    prompt = f"""Ты эксперт, оценивающий ответы в интеллектуальной игре. Оцени ответ:
       true - ответ правильный, false - ответ неправильный.
       
    Вопрос: {question}
    Ответ: {model_answer}

    Правильный ответ: {correct_answer}
    Допустимые варианты ответа, которые тоже засчитываются как верные: {', '.join(zachet) if zachet else 'Точный ответ'}

    Верни JSON:
    {{
    "is_correct": true или false,
    "comment": "Краткое объяснение оценки"
    }}"""
        
    try:
        chat_completion = openai.chat.completions.create(
            model="deepseek-ai/DeepSeek-V3-0324",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"},
            seed=2,
            temperature=0,
            max_tokens=5000
        )
            
        response_text = chat_completion.choices[0].message.content
        json_str = re.search(r'\{.*\}', response_text, re.DOTALL)
        if not json_str:
            raise ValueError("Не найден JSON в ответе")
        
        evaluation = json.loads(json_str.group())
            
    except Exception as e:
            evaluation = {
            "is_correct": False,
            "comment": f"Ошибка при оценке: {str(e)}",
            "raw": response_text
        }
        
    model_results.append({
        "evaluation_model": "DeepSeek-V3",
        "answer_model": model_responses_dict.get(question_id)['model'],
        "question_id": question_id,
        "question": question,
        "model_answer": model_answer,
        "correct_answer": correct_answer,
        "zachet": ', '.join(zachet) if zachet else 'Точный ответ',
        "evaluation": evaluation
    })
        
    time.sleep(1.0)
    

with open('диссертация_разметка/model_7_evaluation.json', 'w', encoding='utf-8') as f:
    json.dump(model_results, f, ensure_ascii=False, indent=2)

print("Оценка всех моделей завершена!")

Оценка всех моделей завершена!


### 8. GPT

In [ ]:
import json
import re
import time
from openai import OpenAI

client = OpenAI(
  api_key="")

with open('random_sample_260.json', 'r', encoding='utf-8') as f:
    correct_data = json.load(f)

with open('диссертация_разметка/answers_260.json', 'r', encoding='utf-8') as f:
    model_responses = json.load(f)

model_responses_dict = {item['id']: item for item in model_responses}

model_results = []

for i, item in enumerate(correct_data, 1):
    question_id = item['id']
    model_response = model_responses_dict.get(question_id)
    
    if not model_response:
        print(f"[{i}/5] Ответ для вопроса с ID {question_id} не найден")
        continue
        
    question = item['Вопрос']
    model_answer = model_response['answer'].strip()
    correct_answer = item['Ответ']
    zachet = item['Зачет'].split(',') if item['Зачет'] else []

    prompt = f"""Ты эксперт, оценивающий ответы в интеллектуальной игре. Оцени ответ:
true - ответ правильный, false - ответ неправильный.

Вопрос: {question}
Ответ: {model_answer}

Правильный ответ: {correct_answer}
Допустимые варианты ответа, которые тоже засчитываются как верные: {', '.join(zachet) if zachet else 'Точный ответ'}

Верни JSON:
{{
  "is_correct": true или false,
  "comment": "Краткое объяснение оценки"
}}"""


    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=3000,
            top_p=1,
            seed=42
        )

        response_text = response.choices[0].message.content.strip()
        json_str = re.search(r'\{.*\}', response_text, re.DOTALL)

        if json_str:
            evaluation = json.loads(json_str.group())
        else:
            raise ValueError("Не удалось извлечь JSON из ответа")

    except Exception as e:
        evaluation = {
            "is_correct": False,
            "comment": f"Ошибка при оценке: {str(e)}",
            "raw": response_text if 'response_text' in locals() else ""
        }

    model_results.append({
        "question_id": question_id,
        "question": question,
        "model_answer": model_answer,
        "correct_answer": correct_answer,
        "zachet": ', '.join(zachet) if zachet else 'Точный ответ',
        "evaluation": evaluation
    })

    time.sleep(1.0) 
    #print(f"Результат оценки: {evaluation['is_correct']} ({evaluation['comment']})")

output_path = 'диссертация_разметка/model_8_evaluation.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(model_results, f, ensure_ascii=False, indent=2)

print(f"Результаты сохранены в: {output_path}")

## Ручная разметка

In [ ]:
import json
import textwrap

def print_separator(title=None, width=80):
    if title:
        print("\n" + f" {title} ".center(width, "="))
    else:
        print("\n" + "=" * width)

def print_formatted(text, title=None, width=80):
    if title:
        print(f"\n{title}:")
        print("-" * len(title))
    for line in textwrap.wrap(text, width=width):
        print(line)

def save_results(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"\nАвтосохранение: результаты сохранены в {filename}")

with open('random_sample_260.json') as f:
    correct_data = json.load(f)

with open('диссертация_разметка/answers_260.json') as f:
    model_responses = json.load(f)

model_responses_dict = {item['id']: item for item in model_responses}
human_evaluations = []
output_file = 'диссертация_разметка/human_evaluation_results_1.json'
autosave_interval = 10  # Сохранять каждые 10 вопросов

print_separator(" ОЦЕНКА ОТВЕТОВ МОДЕЛИ ЧЕЛОВЕКОМ ")
print(f"Инструкция: вводите 1 если ответ правильный, 0 если неправильный")
print(f"Результаты будут автоматически сохраняться каждые {autosave_interval} вопросов")
input("\nНажмите Enter чтобы начать...")

for idx, item in enumerate(correct_data, 1):
    question_id = item['id']
    model_response = model_responses_dict.get(question_id)
    
    if not model_response:
        print(f"\n⚠ Ответ для вопроса с ID {question_id} не найден")
        continue
    
    print_separator(f" ВОПРОС {idx}/{len(correct_data)} ")
    
    print(f"\nID вопроса: {question_id}")
    print_formatted(item['Вопрос'], "Вопрос")
    print_formatted(model_response['answer'].strip(), "Ответ модели")
    print_formatted(item['Ответ'], "Правильный ответ")
    
    if item['Зачет']:
        print_formatted(", ".join(item['Зачет'].split(',')), "Допустимые варианты")
    
    while True:
        try:
            rating = input("\nОценка (1 - верно, 0 - неверно): ").strip()
            rating = int(rating)
            if rating not in [0, 1]:
                raise ValueError
            break
        except ValueError:
            print("Ошибка! Пожалуйста, введите 1 или 0")
    
    comment = input("Комментарий (необязательно): ").strip()
    
    human_evaluations.append({
        "model_name": model_response['model'],
        "question_id": question_id,
        "question": item['Вопрос'],
        "model_answer": model_response['answer'].strip(),
        "correct_answer": item['Ответ'],
        "zachet": item['Зачет'],
        "human_score": bool(rating),
        "human_comment": comment if comment else None,
    })

    # Периодическое сохранение
    if idx % autosave_interval == 0:
        save_results(human_evaluations, output_file)
        print_separator(f" ПРОГРЕСС: {idx}/{len(correct_data)} ")
        input("Нажмите Enter чтобы продолжить...")

save_results(human_evaluations, output_file)

print_separator(" ОЦЕНКА ЗАВЕРШЕНА! ")
print(f"Все результаты сохранены в файл: {output_file}")
print_separator()

## Оценивание экспертов

In [ ]:
human_data = json.load(open("диссертация_разметка/human_evaluation_results.json"))
model_6_data = json.load(open("диссертация_разметка/model_6_evaluation.json"))
model_7_data = json.load(open("диссертация_разметка/model_7_evaluation.json"))
model_8_data = json.load(open("диссертация_разметка/model_8_evaluation.json"))

def compare_with_human(model_data, human_data):
    matches = 0
    total = 0
    
    human_dict = {item["question_id"]: item for item in human_data}
    
    for model_item in model_data:
        q_id = model_item["question_id"]
        if q_id in human_dict:
            human_eval = human_dict[q_id]
            model_is_correct = model_item["evaluation"]["is_correct"]
            human_is_correct = human_eval["human_score"]
            
            if model_is_correct == human_is_correct:
                matches += 1
            total += 1
    
    return matches / total if total > 0 else 0

accuracy_6 = compare_with_human(model_6_data, human_data)
accuracy_7 = compare_with_human(model_7_data, human_data)
accuracy_8 = compare_with_human(model_8_data, human_data)

print(f"Model 6 accuracy: {accuracy_6:.2%}")
print(f"Model 7 accuracy: {accuracy_7:.2%}")
print(f"Model 8 accuracy: {accuracy_8:.2%}")

best_model = max(
    [("model_6", accuracy_6), ("model_7", accuracy_7), ("model_8", accuracy_8)],
    key=lambda x: x[1]
)
print(f"Наиболее согласованная модель: {best_model[0]} ({best_model[1]:.2%})")

Model 6 accuracy: 97.31%
Model 7 accuracy: 99.23%
Model 8 accuracy: 99.62%
Наиболее согласованная модель: model_8 (99.62%)


In [ ]:
def find_disagreements(model_data, human_data, model_name):
    disagreements = []
    human_dict = {item["question_id"]: item for item in human_data}
    
    for model_item in model_data:
        q_id = model_item["question_id"]
        if q_id in human_dict:
            human_eval = human_dict[q_id]
            model_is_correct = model_item["evaluation"]["is_correct"]
            human_is_correct = human_eval["human_score"]
            
            if model_is_correct != human_is_correct:
                disagreements.append({
                    "question_id": q_id,
                    "question": human_eval["question"],
                    "model_answer": model_item["model_answer"],
                    "correct_answer": human_eval["correct_answer"],
                    "model_evaluation": model_is_correct,
                    "human_evaluation": human_is_correct,
                    "model_comment": model_item["evaluation"].get("comment", ""),
                    "human_comment": human_eval.get("human_comment", "")
                })
    
    return disagreements

# Находим расхождения для каждой модели
disagreements_6 = find_disagreements(model_6_data, human_data, "model_6")
disagreements_7 = find_disagreements(model_7_data, human_data, "model_7")
disagreements_8 = find_disagreements(model_8_data, human_data, "model_8")

# Выводим результаты
def print_disagreements(disagreements, model_name):
    print(f"\n=== Расхождения для {model_name} (всего: {len(disagreements)}) ===")
    for item in disagreements:
        print(f"\nID вопроса: {item['question_id']}")
        print(f"Вопрос: {item['question']}")
        print(f"Ответ модели: {item['model_answer']}")
        print(f"Правильный ответ: {item['correct_answer']}")
        print(f"Оценка модели: {item['model_evaluation']} (комментарий: {item['model_comment']})")
        print(f"Оценка человека: {item['human_evaluation']} (комментарий: {item['human_comment']})")

print_disagreements(disagreements_6, "model_6")
print_disagreements(disagreements_7, "model_7")
print_disagreements(disagreements_8, "model_8")


=== Расхождения для model_6 (всего: 7) ===

ID вопроса: 978
Вопрос: Несмотря на то, что ваянг-кулит является ИМ, там используются ярко окрашенные куклы. Назовите ЕГО двумя словами, которые начинаются на одни и те же буквы.
Ответ модели: Теневой Театр
Правильный ответ: театр теней.
Оценка модели: False (комментарий: Правильный ответ — «театр теней». Хотя оба слова в ответе участника начинаются на «т», порядок слов не соответствует общепринятому названию. «Теневой театр» — описательное выражение, а не точный термин.)
Оценка человека: True (комментарий: None)

ID вопроса: 427
Вопрос: Согласно результатам исследования Винанда А. Тилбурга, окружающие выше оценивают интеллектуальные способности и статус людей с ЭТИМ. Какое ЭТО есть в вопросе?
Ответ модели: очками
Правильный ответ: А
Оценка модели: True (комментарий: Ответ верен. Исследование Винанда А. Тилбурга действительно показало, что люди в очках воспринимаются как более интеллектуальные и статусные. Слово «очками» присутствует в вопро

In [ ]:
from sklearn.metrics import cohen_kappa_score

def calculate_kappa(model_data, human_data):
    human_dict = {item["question_id"]: item for item in human_data}
    
    model_scores = []
    human_scores = []
    
    for model_item in model_data:
        q_id = model_item["question_id"]
        if q_id in human_dict:
            human_eval = human_dict[q_id]
            model_scores.append(model_item["evaluation"]["is_correct"])
            human_scores.append(human_eval["human_score"])
    
    if len(model_scores) > 0:
        kappa = cohen_kappa_score(human_scores, model_scores)
        return kappa
    else:
        return None

# Считаем каппу для каждой модели
kappa_6 = calculate_kappa(model_6_data, human_data)
kappa_7 = calculate_kappa(model_7_data, human_data)
kappa_8 = calculate_kappa(model_8_data, human_data)

print(f"Каппа Коэна для model_6: {kappa_6:.3f}")
print(f"Каппа Коэна для model_7: {kappa_7:.3f}")
print(f"Каппа Коэна для model_8: {kappa_8:.3f}")

Каппа Коэна для model_6: 0.858
Каппа Коэна для model_7: 0.959
Каппа Коэна для model_8: 0.979
